# PDFからチャンク化する事例
セクション分けされているPDFデータを章節条にセクショニングでチャンク化したデータに変換するサンプル

In [ ]:
import fitz  # PyMuPDF
import json
import re


# -----------------------------------------------------------------------------
# PDFからテキストを抽出
# -----------------------------------------------------------------------------
def pdf2text(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    md_lines = []
    for page in doc:
        text = page.get_text("text")
        lines = text.split('\n')

        for line in lines:
            line = line.strip()
            if not line:
                continue
            md_lines.append(line)
        # ページの区切り
        md_lines.append("")
    return '\n'.join(md_lines)


# -----------------------------------------------------------------------------
# 全角文字を半角文字で正規化
# -----------------------------------------------------------------------------
def normalize_text(text: str):
    return text.translate(str.maketrans(
        {chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}
    ))


# -----------------------------------------------------------------------------
# 文章をセクション・箇条書き単位で
# -----------------------------------------------------------------------------
def normalize_article(text: str):
    # 〇箇条書きとセクションで区切り箇所を特定て置換
    #  ※分割箇所を"---"で表現
    # 正規表現にてセクション、箇条書きを特定し、---(仮)で分割
    # ・第<数値>で始まる・・・第N章 ~~、第N節 ~~、第N条 ~~など
    # ・<数値>.で始まる・・・ 1. ~~、2. ~~
    # ・(数値)で始まる・・・ (1) ~~、 (2) ~~
    text = re.sub(
        r"^(\s*第\d+.*?|\s*\d+\..*|\s*\(\d+\).*)$",
        r"---\1", text, flags=re.MULTILINE)

    # 〇元データの改行を削除
    # 元の改行は座標位置に依存するため
    text = re.sub(r"\r?\n", "", text)

    # 〇改行を再設定
    # 分割箇所("---")を再度改行に復元
    text = re.sub(r"---", "\n", text)
    # 句点（。）で改行
    text = re.sub(r"。", "。\n", text)

    # 一文の前後余白等を削除
    text = "\n".join(
        line.strip() for line in text.splitlines()
        if line.strip()
    )

    # 〇セクションに更に改行を増やして強調
    # わかりやすくするため更に改行を追加
    text = re.sub(
        r"^(第\s*\d+\s*章.*)$",
        r"\n\n\1", text, flags=re.MULTILINE
    )
    text = re.sub(
        r"^(第\s*\d+\s*条.*)$",
        r"\n\1", text, flags=re.MULTILINE
    )
    text = re.sub(
        r"^(第\s*\d+\s*条.*)$",
        r"\n\1", text, flags=re.MULTILINE
    )
    return text


def chunk_articles(text: str):
    articles = []
    section = ""  # 現在の章
    chapter = ""  # 現在の節
    article = ""  # 現在の条
    buffer = {
        "chapter": "",
        "section": "",
        "article": ""
    }

    # バッファ
    changed = False

    # 条単位でデータを区切る
    for line in text.splitlines():

        # 書込みフラグが有効な場合、書込
        if changed:
            changed = False
            if buffer["article"]:
                articles.append(buffer)
            buffer = {
                "chapter": chapter,
                "section": section,
                "article": article
            }

        # 〇条の場合
        if re.match(r"^(第\s*\d+\s*条.*)$", line):
            changed = True
            article = line

        # 〇節の場合
        # 現在の節を更新
        elif re.match(r"^(第\s*\d+\s*節.*)$", line):
            changed = True
            article = ""
            section = line

        # 〇第N章、または、付則の場合
        # 現在の章と節を更新
        elif re.match(r"^(第\s*\d+\s*章.*|付則.*)$", line):
            changed = True
            article = ""
            section = ""
            chapter = line

        # 区切り以外の文の場合、bufferに追記
        elif line.strip():
            # 区切り以外の文字は条として書き込む
            buffer["article"] = f"{buffer["article"]}{line}\n"

    # 残りのバッファを書き込み
    articles.append(buffer)

    return articles


def convert_pdf_to_json(input_path: str, output_path):
    with open(output_path, mode="w", encoding="utf8") as f:
        # テキスト読込
        text = pdf2text(input_path)
        # テキスト正規化
        text = normalize_text(text)
        text = normalize_article(text)
        # チャンク化
        articles = chunk_articles(text)
        # データ書込み
        f.write(json.dumps(articles, ensure_ascii=False))


if __name__ == "__main__":
    convert_pdf_to_json(input_path="規則.pdf", output_path="就業規則.json")